In [1]:
library(SpatialPCA)
library(slingshot)

Loading required package: princurve

Loading required package: TrajectoryUtils

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLog

In [2]:
exp_df <- read.csv("simulated_expression.csv", row.names = 1)

In [4]:
rownames(exp_df) <- gsub("_", "-", rownames(exp_df))

In [5]:
head(exp_df)

,cell1,cell2,cell3,cell4,cell5,cell6,cell7,cell8,cell9,cell10,⋯,cell491,cell492,cell493,cell494,cell495,cell496,cell497,cell498,cell499,cell500
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Burn1-TF1,8,9,8,6,6,10,3,14,7,2,⋯,6,9,13,8,11,6,10,12,2,3
Burn1-TF2,13,17,3,11,9,11,3,14,7,4,⋯,8,8,20,9,12,9,13,5,6,8
Burn1-TF3,17,9,15,11,11,16,14,16,10,6,⋯,9,15,21,13,21,16,16,16,11,12
Burn2-TF1,16,10,10,18,16,14,10,17,20,13,⋯,12,18,23,17,15,12,19,25,15,17
Burn3-TF1,15,11,9,4,21,12,5,11,7,5,⋯,4,7,6,14,10,10,9,15,5,5
Burn3-TF2,7,3,3,2,11,10,2,2,7,8,⋯,4,12,7,8,7,5,15,13,3,5


In [ ]:
meta_info_df <- read.csv("simulation_linear_ontrac_input.csv", row.names = 1)

In [8]:
xy_coods <- as.matrix(meta_info_df[colnames(exp_df),c('x', 'y')])

In [9]:
head(xy_coods)

,x,y
cell1,0.815010259,0.39285904
cell2,0.007831625,0.52816467
cell3,0.238569948,0.28056903
cell4,0.954609492,0.95035290
cell5,0.008683761,0.04630861
cell6,0.624925804,0.07620596


In [ ]:
# no enough spatial genes detected here, so SpatialPCA can not get results

spatialPCA_obj <- CreateSpatialPCAObject(counts=as.matrix(exp_df), location=xy_coods, project = "SpatialPCA",gene.type="spatial",sparkversion="spark",numCores_spark=5,gene.number=3000, customGenelist=NULL,min.loctions = 2, min.features=2)


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


## Use SCTransform function in Seurat to normalize data. 


Running SCTransform on assay: RNA

Running SCTransform on layer: counts

vst.flavor='v2' set. Using model with fixed slope and excluding poisson genes.

`vst.flavor` is set to 'v2' but could not find glmGamPoi installed.
Please install the glmGamPoi package for much faster estimation.
--------------------------------------------
install.packages('BiocManager')
BiocManager::install('glmGamPoi')
--------------------------------------------
Falling back to native (slower) implementation.


Variance stabilizing transformation of count matrix of size 100 by 500

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 100 genes, 500 cells

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in sqrt(1/i):
“NaNs produced”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in sqrt(1/i):
“NaNs 

## Use spark.test function in SPARK package to select spatially variable genes. 
## ===== SPARK INPUT INFORMATION ==== 
## number of total samples:  500 
## number of total features:  98 
## number of adjusted covariates:  0 
# fitting normalized count-based spatial model under the null hypothesis ... 
## testing Gaussian kernel: 1...
## testing Periodic kernel: 1...
## testing Gaussian kernel: 2...
## testing Periodic kernel: 2...
## testing Gaussian kernel: 3...
## testing Periodic kernel: 3...
## testing Gaussian kernel: 4...
## testing Periodic kernel: 4...
## testing Gaussian kernel: 5...
## testing Periodic kernel: 5...
## Identified  1  spatial genes through spark.test function. 
The  number of significant spatial genes is less than the specified number of spatial genes. 
## Using  1  significant spatially variable genes. 


In [15]:
spatialPCA_obj <- SpatialPCA_buildKernel(spatialPCA_obj, kerneltype="gaussian", bandwidthtype="SJ",bandwidth.set.by.user=NULL)
spatialPCA_obj <- SpatialPCA_EstimateLoading(spatialPCA_obj,fast=FALSE,SpatialPCnum=10) 
spatialPCA_obj <- SpatialPCA_SpatialPCs(spatialPCA_obj, fast=FALSE)

## Selected kernel type is:  gaussian  


ERROR: Error in 1:dim(object@normalized_expr)[1]: argument of length 0


In [ ]:
clusterlabel= walktrap_clustering(clusternum=6,
                                  latent_dat=spatialPCA_obj@SpatialPCs,
                                  knearest=round(sqrt(dim(spatialPCA_obj@SpatialPCs)[2]))
                                 ) 

In [ ]:
sce <- SingleCellExperiment(assays = as.matrix(exp_df))

In [ ]:
reducedDims(sce) <- SimpleList(DRM = t(spatialPCA_obj@SpatialPCs))

In [ ]:
colData(sce)$clusterlabel <- factor(clusterlabel)    
sce  <-slingshot(sce, clusterLabels = 'clusterlabel', reducedDim = 'DRM',start.clus="3" ) 
# in this data we set white matter region as start cluster, one can change to their preferred start region 

# summary(sce@colData@listData)
pseudotime_traj1 <- sce@colData@listData$slingPseudotime_1 # in this data only one trajectory was inferred


In [ ]:
write.csv(pseudotime_traj1, "pseudotime_spatial_pca.csv")